In [1]:
!wandb login 86c64a2b2da45013d9c4f3c597d330492016a846

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pl_bolts.datamodules import MNISTDataModule
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import torchvision
from torchmetrics.functional import accuracy
import wandb

In [7]:
def create_model():
    model = torchvision.models.resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(1, 64, 3, 1, 3, bias=False)
    model.maxpool = nn.Identity()
    return model

In [8]:
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, X):
        return self.model(X)

    def configure_optimizers(self):
        return torch.optim.RAdam(self.parameters())
        
    def evaluate(self, batch, stage):

        X, y = batch
        logits = self(X)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.functional.accuracy(preds, y, task='multiclass', num_classes=10)

        self.log(f"{stage}_loss", loss)
        self.log(f"{stage}_acc", accuracy)

        return loss, accuracy

    def training_step(self, batch, _):
        loss, _ = self.evaluate(batch, "train")
        return loss

    def validation_step(self, batch, _):
        self.evaluate(batch, "val")

    def test_step(self, batch, _):
        self.evaluate(batch, "test")

In [9]:
wandb.finish()
pl.seed_everything(0)
wandb_logger = WandbLogger()
trainer = pl.Trainer(
    max_epochs=20,
    accelerator='gpu',
    devices=-1,
    logger=wandb_logger)
model = MNISTModel()
dm = MNISTDataModule(data_dir='MNIST/', val_split=.1, num_workers=8, normalize=True)
trainer.fit(model, dm)
wandb.finish()

Global seed set to 0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.691    Total estimated model params size (MB)


Epoch 19: 100%|████| 1876/1876 [02:50<00:00, 11.01it/s, loss=0.0266, v_num=vuea]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|████| 1876/1876 [02:50<00:00, 11.00it/s, loss=0.0266, v_num=vuea]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_acc,▁█▁█▅▁████▅█▅███████████████████████████
train_loss,▆▂▇▂▂█▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▆▇▇▇▇▇▇▇█▇▇█▇██████
val_loss,█▃▂▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▁▁
epoch,19
train_acc,1.0
train_loss,2e-05
trainer/global_step,33759
val_acc,0.993
